# <span style="font-family: 'Computer Modern'; font-size: 42pt; font-weight: bold;">Quantum Convolutional Neural Network (QCNN) Using *PennyLane*</span>

***

In [1]:
### IMPORTS / DEPENDENCIES:

# PennyLane:
import pennylane as qml
from pennylane import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as patches # Quantum Circuit Drawings
mpl.rcParams.update(mpl.rcParamsDefault)
from tqdm import tqdm
import csv

import torch
import math
import random

from scipy.linalg import expm # Unitary-Related Operations

In [ ]:
### PACKAGE IMPORTS (UN-COMMENT WHEN COMPLETED)

## gellmann_ops.py
#from gellmann_ops import GellMannOps as gell_ops # GELL MANN MATRIX OPERATION CLASS
#from lppc_qcnn.gellmann_ops import ParamOps as param_ops # PARAMETER OPERATIONS HELPER CLASS

## qc_data.py
# from lppc_qcnn.qc_data import DataLPPC as lppc_data # MNIST DATASET CLASS

## qcircuit.py
# from lppc_qcnn.qcircuit import QCircuitLPPC as qc_circ # QUANTUM CIRCUIT AND LAYERS CLASS
# from lppc_qcnn.qcircuit import OptStepLPPC as opt_lppc # OPTIMIZATION AND COST CLASS

***

<span style="font-family: 'Computer Modern'; font-weight: bold; font-size: 26pt;">THE MNIST DATASET</span>

<img src="qcnn-figures/mnist_plot.png" alt="MNIST Dataset Sample Images" style="display: block; margin-left: auto; margin-right: auto; width: 80%;">

<p style="text-align: center; font-family: 'Computer Modern', serif;">
    Sample of the handwritten digital pixelations from the MNIST dataset, which are used for training and testing the QCNN model.<br>
    <em>Image source: <a href="https://corochann.com/mnist-dataset-introduction-532/">https://corochann.com/mnist-dataset-introduction-532/</a></em>
</p>

<span style="font-family: 'Computer Modern'; font-size: 16pt; font-weight: bold;">Loading the MNIST Dataset:</span>

<span style="font-family: 'Computer Modern'; font-size: 14pt;">For our QCNN, we load the MNIST dataset using TorchVision, which allows us to process the data with quantum features and pass it into our neural network. We define the path for the MNIST data directory below, and use TorchVision to load in the MNIST dataset (Note:  the exact "path name" that you choose can be arbitrary and/or at your discretion, as our dataloaders will be able to handle the data loading under most root name cases). We then initialize the batch sizes for the MNIST training and testing data sets. In this model, we set the batch size for the training data at 350, and at 250 for the testing data.</span>

In [ ]:
# Import relevant classclass(es) for MNIST DATA LOADING AND PROCESSING before passing data to QC:
from lppc_qcnn.qc_data import DataLPPC as lppc_data

In [ ]:
### READING AND LOADING DATA: 

# Set directory for data:
data_path = './DATA'

# Set batch sizes for training and testing data:
batch_train_qcnn = 350
batch_test_qcnn = 250

# Note: Selections of batch_train=350 and batch_test=250 were chosen for our own preferred sample size, and is
# also up to your own discretion.
train_images, train_labels, test_images, test_labels = lppc_data.load_mnist_torch(batch_train=batch_train_qcnn,
                                                                    batch_test=batch_test_qcnn, root=data_path)

# Print relevant shapes and types of your training and testing data to check progress:
print(f"train_images shape: {train_images.shape}, dtype: {train_images.dtype}")
print(f"test_images shape: {test_images.shape}, dtype: {test_images.dtype}")
print(f"train_labels shape: {train_labels.shape}, dtype: {train_labels.dtype}")
print(f"test_labels shape: {test_labels.shape}, dtype: {test_labels.dtype}")

<span style="font-family: 'Computer Modern'; font-size: 16pt; font-weight: bold;">MNIST DATA TRANSFORMATIONS:</span>

<span style="font-family: 'Computer Modern'; font-size: 14pt;">We initialize the reduction sizes for the MNIST training and testing data sets. In this model, we set the reduction size for the training data at 500, and at 100 for the testing data. We then # reduce the number of data points in the training and testing datasets as necessary (Note: it is important to ensure that at least one of the specified reduction values for "n_train" and "n_test" is smaller than its  corresponding batch size values used during the loading step for the MNIST data, or else no reduction stage is necessary in the steps for the model).</span>

In [ ]:
### REDUCING THE IMPORTED MNIST DATA

# Reduction sizes:
n_train_qcnn = 500
n_test_qcnn = 100

# Reduce datasets as needed:
if n_train_qcnn < batch_train_qcnn or n_test_qcnn < batch_test_qcnn:
    train_images, train_labels, test_images, test_labels = lppc_data.mnist_reduce(train_images, train_labels,
                                        test_images, test_labels, n_train=n_train_qcnn, n_test=n_test_qcnn)

# Print relevant shapes and types of your training and testing data to check progress:
print(f"train_images shape: {train_images.shape}, dtype: {train_images.dtype}")
print(f"test_images shape: {test_images.shape}, dtype: {test_images.dtype}")
print(f"train_labels shape: {train_labels.shape}, dtype: {train_labels.dtype}")
print(f"test_labels shape: {test_labels.shape}, dtype: {test_labels.dtype}")

In [ ]:
### FLATTENING THE IMPORTED MNIST DATA


# TODO
train_images, test_images = lppc_data.mnist_flatten(train_images, test_images)

# Print relevant shapes and types of your training and testing data to check progress:
print(f"train_images shape: {train_images.shape}, dtype: {train_images.dtype}")
print(f"test_images shape: {test_images.shape}, dtype: {test_images.dtype}")

In [ ]:
### PADDING THE FLATTENED DATASETS


# TODO
x_train, y_train, x_test, y_test = lppc_data.mnist_padding(train_images, train_labels,
                                                           test_images, test_labels)

# Print relevant shapes and types of your training and testing data to check progress:
print(f"x_train shape: {x_train.shape}, dtype: {x_train.dtype}")
print(f"x_test shape: {x_test.shape}, dtype: {x_test.dtype}")
print(f"y_train shape: {y_train.shape}, dtype: {y_train.dtype}")
print(f"y_test shape: {y_test.shape}, dtype: {y_test.dtype}")

***

<span style="font-family: 'Computer Modern'; font-weight: bold; font-size: 18pt;">QCNN MODEL</span>

In [ ]:
# Import relevant classclass(es) for QUANTUM CIRCUIT (before passing weights to the QC):
from gellmann_ops import GellMannOps as gell_ops # GELL MANN MATRIX OPERATION CLASS
from lppc_qcnn.gellmann_ops import ParamOps as param_ops # PARAMETER OPERATIONS HELPER CLASS

from lppc_qcnn.qcircuit import QCircuitLPPC as qc_circ # QUANTUM CIRCUIT AND LAYERS CLASS

<span style="font-family: 'Computer Modern'; font-size: 14pt;">_Trainable Parameters_:</span>

In [1]:
### INITIALIZING QUBIT PARAMETERS

# Iniitialize the number of qubits to use within the QCNN. Note that his model is a 10-qubit system.
n_qubits = 10  # Number of qubits
active_qubits = 10 # Number of active qubits (same as n_qubits, tracks QC operations)

In [ ]:
### INITIALIZING WEIGHTS

# TODO
qcnn_weights0 = np.random.uniform(0, np.pi, size=(n_qubits, 1, 3))

# TODO
qcnn_weights = param_ops.broadcast_params(qcnn_weights0)

# Print relevant shapes and types of weights array and sample weight:
print(f"qcnn_weights shape: {qcnn_weights[0].shape}, dtype: {type(qcnn_weights[0])}")
print(f"qcnn_weights shape: {qcnn_weights.shape}, dtype: {qcnn_weights.dtype}")

<span style="font-family: 'Computer Modern'; font-size: 14pt;">_Circuit Construction_:</span>

In [ ]:
### CONVOLUTIONAL LAYER:

# TODO


In [ ]:
### POOLING LAYER:

# TODO


In [ ]:
### FULLY CONNECTED LAYER:

# TODO


In [ ]:
### QUANTUM CIRCUIT:

# TODO


***

<span style="font-family: 'Computer Modern'; font-weight: bold; font-size: 18pt;">TRAINING / OPTIMIZATION </span>

In [ ]:
# Import relevant class(es) for TRAINING AND OPTIMIZATION-RELATED PROCESSES prior to training weights:
from lppc_qcnn.qcircuit import OptStepLPPC as opt_lppc # OPTIMIZATION AND COST CLASS
# from lppc_qcnn.qcircuit import QCircuitLPPC as qc_circ

<span style="font-family: 'Computer Modern'; font-size: 16pt;">_Preparation_:</span>

In [ ]:
### TRAINING PREPARATION

# Initialize the selected optimizer (Note: in this model, the Stochastic Gradient Descent (SGD) Optimizer was 
# determined to be the most suitable, although the choice of optimizer is additionally up to your own discretion.)

# Set value to 1, 2, or 3 based on desired optimizer selection from 'opt' (Note: For this model, "1" corresponds 
# to the Stochastic Gradient Descent (SGD) Optimizer. You can use qc_opt_print() to see all available
# optimizers to choose from.
opt_num_lppc = 1 # TAKE AS PARAMETER

# List of all available / acceptable optimizers for QCNN model:
# {1: qml.GradientDescentOptimizer,
#      2: qml.AdamOptimizer,
#      3: qml.COBYLAOptimizer
# }

# Select Stochastic Gradient Descent (SGD) Optimizer:
opt = qc_opt_select(opt_num_lppc)

# Initialize important training parameters:
learning_rate = 0.1
batch_size = 10
max_iter = 100
conv_tol = 1e-06

num_steps = 10
loss_history = []

<span style="font-family: 'Computer Modern'; font-size: 16pt;">_Training Model_:</span>

In [ ]:
### OPTIMIZATION AND TRAINING


# Training Loop:
for step in range(num_steps):
    qcnn_weights, loss = opt_lppc.stoch_grad_V2(opt, opt_lppc.mse_cost, qcnn_weights, x_train, y_train,
                                                learning_rate, batch_size, max_iter, conv_tol)
    
    loss_history.append(loss)  # Accumulate loss

    # Print step and cost:
    print(f"Step {step}: cost = {loss}")

# Evaluate Optimization Accuracy on testing dataset:
predictions = np.array([qc_circ.q_circuit_V3(qcnn_weights, xi, active_qubits) for xi in x_test])

<span style="font-family: 'Computer Modern'; font-size: 14pt;">_Accuracy_:</span>

In [ ]:
### PREDICTIONS

# Calculate and determine accuracy of the QCNN model:
accuracy = opt_lppc.accuracy_V1(predictions, y_test)
print(f"Model accuracy: {accuracy * 100:.2f}%")

***

***

<span style="font-family: 'Computer Modern'; font-weight: bold; font-size: 20pt;">_APPENDIX_</span>

In [ ]:
# TODO

***

<p style="font-family: 'Computer Modern'; font-size: 10pt; font-weight: bold; text-align: center;">
    © The Laboratory for Particle Physics and Cosmology (LPPC) at Harvard University, Cambridge, MA<br>
    © Sean Chisholm<br>
    © Pavel Zhelnin
</p>

***